In [1]:
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from argparse import Namespace

from torch import nn
from dataloader import load_data
from tqdm import tqdm


train_inputs,train_lables,test_inputs,test_lables=load_data(60000,60000,device=torch.device('cuda:5'))




load data set
create loader
construct data


100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


In [ ]:
import torch
import model
# from model import Bias
from torch import nn
import importlib
importlib.reload(model)


def acc(model):
    model.eval()
    correct=0
    total=0
    with torch.no_grad():
        
        for i in range(len(test_inputs)):
            outputs=model(test_inputs[i])
            _,predicted=torch.max(outputs.data,1)
            total+=len(predicted)
            correct+=(predicted==test_lables[i]).sum().item()
    model.train()
    return correct/total


mlp = torch.load('./mlp.pth')
print(len(mlp.layers))
device=next(mlp.parameters()).device

for k,layer in enumerate(mlp.layers):
    if (isinstance(layer, torch.nn.Linear)):
        w=layer.weight
        b=layer.bias
        U,S,V=torch.linalg.svd(w)
        print(U)
        nU=U.shape[0]
        mU=U.shape[1]
        nV=V.shape[0]
        mV=V.shape[1]
        nS=mU
        mS=nV
        S_diag = torch.zeros(nS,mS)
        for i in range(S.shape[0]):
            S_diag[i, i] = S[i]
        S=S_diag

        newlinear1=torch.nn.Linear(nU,mU,bias=False).to(device)
        newlinear2=torch.nn.Linear(nS,mS,bias=False).to(device)
        newlinear3=torch.nn.Linear(nV,mV,bias=False).to(device)
        newlinear1.weight=nn.Parameter(U)
        newlinear2.weight=nn.Parameter(S)
        newlinear3.weight=nn.Parameter(V)
        newbias=model.Bias(b)
        newbias=newbias.to(device)
        svded=nn.Sequential(newlinear3,newlinear2,newlinear1,newbias)
        mlp.layers[k]=svded

mlp.eval()
mlp.to(device)
for layer in mlp.layers:
    if(isinstance(layer,torch.nn.Sequential)):
        for sublayer in layer:
            sublayer.to(device)
            if(not isinstance(sublayer,nn.Module)):
                print(sublayer.weight.shape)
    print('----------------')

print(acc(mlp))


/tmp/ipykernel_1519498/197572307.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mlp = torch.load('./mlp.pth')


15
tensor([[ 0.0021,  0.0352,  0.0155,  ...,  0.0420,  0.0046,  0.0068],
        [-0.0354, -0.0184, -0.0227,  ..., -0.0311, -0.0369,  0.0378],
        [-0.0101, -0.0039,  0.0147,  ..., -0.0220, -0.0107, -0.0076],
        ...,
        [-0.0097, -0.0063, -0.0074,  ...,  0.2867, -0.0310, -0.0026],
        [-0.0365, -0.0175, -0.0040,  ...,  0.0108,  0.2454,  0.0023],
        [ 0.0231,  0.0455,  0.0373,  ...,  0.0011, -0.0090,  0.2486]],
       device='cuda:5', grad_fn=<LinalgSvdBackward0>)
tensor([[ 0.0592, -0.0850,  0.0028,  ...,  0.0030, -0.0024, -0.0034],
        [-0.0401, -0.0265, -0.0410,  ...,  0.0113,  0.0107, -0.0168],
        [ 0.0226, -0.0419,  0.0148,  ..., -0.0375, -0.0224, -0.0699],
        ...,
        [ 0.0056, -0.0447,  0.0185,  ...,  0.0177, -0.0181, -0.0476],
        [ 0.0067, -0.0162, -0.0211,  ..., -0.0392, -0.0498, -0.0117],
        [-0.0233, -0.0213,  0.0196,  ...,  0.0291, -0.0618, -0.0314]],
       device='cuda:5', grad_fn=<LinalgSvdBackward0>)
tensor([[-0.0346,  0.